In [1]:
import networkx as nx

In [2]:
brain = nx.read_edgelist("/content/bn-mouse-kasthuri_graph_v4.edges")

## Numero di nodi e archi




In [4]:
len(brain.nodes)

1029

In [5]:
len(brain.edges)

1559

## Density

In [7]:
density = nx.density(brain)
density

0.0029475937123042656

Non è sparsa quanto un social network ma è molto sparsa.